In [1]:
import pandas as pd
import sqlite3
import re
import os

# Set display options
pd.set_option('display.float_format', '{:.6f}'.format)

# Definir rutas de archivos y directorios
ruta = os.getcwd()
ruta_data = os.path.abspath(os.path.join(ruta, '..', 'data'))
file_obligaciones = os.path.join(ruta_data, 'datos_crudos', 'obligaciones_clientes.xlsx')
file_tasas = os.path.join(ruta_data, 'datos_crudos', 'tasas_productos.xlsx')
database_path = os.path.join(ruta_data, 'datos_refinados', 'database.db')
sql_file_path = 'consulta_obligaciones.sql'

# Leer los archivos Excel de obligaciones y tasas
df_obligaciones = pd.read_excel(file_obligaciones, sheet_name=0)
df_tasas = pd.read_excel(file_tasas, sheet_name=0)

# Redondear columnas numéricas en df_tasas a 6 decimales
numerical_columns = df_tasas.select_dtypes(include=['float', 'int']).columns
df_tasas[numerical_columns] = df_tasas[numerical_columns].round(6)

# Crear una conexión a la base de datos SQLite
conn = sqlite3.connect(database_path)

# Escribir los DataFrames en SQLite, especificando tipo 'REAL' para columnas numéricas
df_obligaciones.to_sql('obligaciones', conn, if_exists='replace', index=False)
df_tasas.to_sql('tasas', conn, if_exists='replace', index=False, dtype={col: 'REAL' for col in numerical_columns})

# Definir la función personalizada para extraer el nombre del producto
def extraer_nombre_producto(id_producto):
    if id_producto is None:
        return 'desconocido'
    s = str(id_producto).strip()
    # Normalizar guiones
    s = s.replace('–', '-').replace('—', '-')
    if '-' in s:
        # Extraer texto después del último guión
        parts = s.rsplit('-', 1)
        substring = parts[1].strip()
        # Eliminar caracteres no alfabéticos al principio
        substring = re.sub(r'^[^a-zA-Z]+', '', substring)
        #  Obtener la primera palabra
        first_word = substring.split(' ')[0]
        return first_word.lower()
    else:
        return 'desconocido'

# Registrar la función personalizada en SQLite
conn.create_function("extraer_nombre_producto", 1, extraer_nombre_producto)

# Leer y ejecutar el script SQL
with open(sql_file_path, 'r') as sql_file:
    sql_script = sql_file.read()
    cursor = conn.cursor()
    cursor.executescript(sql_script)

# Confirmar cambios y cerrar la conexión
conn.commit()
conn.close()

print("Proceso completado. La base de datos SQLite ha sido actualizada con las nuevas tablas y cálculos.")

Proceso completado. La base de datos SQLite ha sido actualizada con las nuevas tablas y cálculos.
